In [1]:
import sys
from stocktick import StockTick
#from pyspark import SparkContext

In [11]:
### TODO: Return a StockTick object with the maximum value between a and b for each one of its
### four fields (price, bid, ask, units)
def maxValuesReduce(a, b):
    maxTick = StockTick(None, "", "", max(a.price,b.price), max(a.bid,b.bid), max(a.ask,b.ask), max(a.units,b.units))
    return maxTick


In [12]:
### TODO: Return a StockTick object with the minimum value between a and b for each one of its
### four fields (price, bid, ask, units)
def minValuesReduce(a, b):
    minTick = StockTick(None, "", "", min(a.price,b.price), min(a.bid,b.bid), min(a.ask,b.ask), min(a.units,b.units))
    return minTick

In [ ]:
def generateSpreadsDailyKeys(tick): ### TODO: Write Me (see below)
    date = tick.date.split("/")
    key = date[2]+"-"+date[0]+"-"+date[1]
    spread = float(tick.ask-tick.bid)/(2*(tick.ask+tick.bid))
    return (key,(tick.time,spread,1))
def generateSpreadsHourlyKeys(tick): ### TODO: Write Me (see below)
    date = tick.date.split("/")
    time = tick.time.split(":")
    key = date[2]+"-"+date[0]+"-"+date[1]+":"+time[0]
    spread = float(tick.ask-tick.bid)/(2*(tick.ask+tick.bid))
    return (key,(tick.time,spread,1))
def spreadsSumReduce(a, b): ### TODO: Write Me (see below)
    if a[0] != b[0]:
        return (b[0],a[1]+b[1],a[2]+b[2])
    else:
        return a

In [2]:
rawTickData = sc.textFile("file:///home/cloudera/Downloads/WDC_tickbidask.txt")

In [3]:
### TODO: use map to convert each line into a StockTick object
tickData =  rawTickData.map(lambda line : StockTick(line))

In [ ]:
tickData.count()

In [4]:
tickData.take(5)

[<stocktick.StockTick instance at 0x7fb07192ad88>,
 <stocktick.StockTick instance at 0x7fb07192add0>,
 <stocktick.StockTick instance at 0x7fb07192ae18>,
 <stocktick.StockTick instance at 0x7fb07192ae60>,
 <stocktick.StockTick instance at 0x7fb07192aea8>]

In [5]:
### TODO: use filter to only keep records for which all fields are > 0
goodTicks = tickData.filter(lambda tick : tick.price > 0 and tick.bid > 0 and tick.ask > 0 and tick.units > 0 )

In [ ]:
goodTicks.count()

In [6]:
## TODO: store goodTicks in the in-memory cache
goodTicks.cache()

PythonRDD[3] at RDD at PythonRDD.scala:43

In [7]:
### TODO: count the number of lines in this RDD
numTicks = goodTicks.count()

In [8]:
numTicks

44602096

In [9]:
### TODO: use goodTicks.reduce(lambda a,b: StockTick(???)) to sum the price, bid, ask and unit fields
    
sumValues = goodTicks.reduce(lambda a,b: StockTick(None,"","",a.price+b.price,a.bid+b.bid,a.ask+b.ask,a.units+b.units))

In [10]:
sumValues.price

2449371535.6641703

In [13]:
maxValuesReduce = goodTicks.reduce(maxValuesReduce) ### TODO: write the maxValuesReduce function
minValuesReduce = goodTicks.reduce(minValuesReduce) ### TODO: write the minValuesReduce function


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 47 in stage 3.0 failed 1 times, most recent failure: Lost task 47.0 in stage 3.0 (TID 163, localhost): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 160405 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1431)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1419)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1418)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:47)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1418)
	

In [ ]:
avgUnits = sumValues.units / float(numTicks)
avgPrice = sumValues.price / float(numTicks)
print "Max units %i, avg units %f\n" % (maxValuesReduce.units, avgUnits)
print "Max price %f, min price %f, avg price %f\n"
            % (maxValuesReduce.price, minValuesReduce.price, avgPrice)

In [ ]:
### Daily and hourly spreads
# Here is how the daily spread is computed. For each data point, the spread can be calculated
# using the following formula : (ask - bid) / 2 * (ask + bid)
# 1) We have a MapReduce phase that uses the generateSpreadsDailyKeys() function as an argument
# to map(), and the spreadsSumReduce() function as an argument to reduce()
# - The keys will be a unique date in the ISO 8601 format (so that sorting dates
# alphabetically will sort them chronologically)
# - The values will be tuples that contain adequates values to (1) only take one value into
# account per second (which value is picked doesn’t matter), (2) sum the spreads for the
# day, and (3) count the number of spread values that have been added.
# 2) We have a Map phase that computes thee average spread using (b) and (c)
# 3) A final Map phase formats the output by producing a string with the following format:
# "<key (date)>, <average_spread>"
# 4) The output is written using .saveAsTextFile("WDC_daily")
avgDailySpreads = goodTicks.map(generateSpreadsDailyKeys).reduceByKey(spreadsSumReduce); # (1)
    #avgDailySpreads = avgDailySpreads.map(lambda a: ???) # (2)
avgDailySpreads = avgDailySpreads.map(lambda (key,a): (key,a[1]/a[2]))
    #avgDailySpreads = avgDailySpreads.sortByKey().map(lambda a: ???) # (3)
avgDailySpreads = avgDailySpreads.sortByKey().map(lambda (key,a): key+","+str(a))
avgDailySpreads = avgDailySpreads.saveAsTextFile("WDC_daily") # (4)
    # For the hourly spread you only need to change the key. How?
avgHourlySpreads = goodTicks.map(generateSpreadsHourlyKeys).reduceByKey(spreadsSumReduce); # (1)
    #avgHourlySpreads = avgHourlySpreads.map(lambda a: ???) # (2)
avgDailySpreads = avgDailySpreads.map(lambda (key,a): (key,a[1]/a[2]))
    #avgHourlySpreads = avgHourlySpreads.sortByKey().map(lambda a: ???) # (3)
avgDailySpreads = avgDailySpreads.sortByKey().map(lambda (key,a): key+","+str(a))
avgHourlySpreads = avgHourlySpreads.saveAsTextFile("WDC_hourly") # (4)
sc.stop()

In [ ]:
if __name__ == "__main__":
"""
Usage: stock
"""
    sc = SparkContext(appName="StockTick")
    # rawTickData is a Resilient Distributed Dataset (RDD)
    rawTickData = sc.textFile("/cs/bigdata/datasets/WDC_tickbidask.txt")
    tickData = ### TODO: use map to convert each line into a StockTick object
    goodTicks = ### TODO: use filter to only keep records for which all fields are > 0
    ### TODO: store goodTicks in the in-memory cache
    numTicks = ### TODO: count the number of lines in this RDD
    sumValues = ### TODO: use goodTicks.reduce(lambda a,b: StockTick(???)) to sum the price, bid,
    ### ask and unit fields
    maxValuesReduce = goodTicks.reduce(maxValuesReduce) ### TODO: write the maxValuesReduce function
    minValuesReduce = goodTicks.reduce(minValuesReduce) ### TODO: write the minValuesReduce function

    avgUnits = sumValues.units / float(numTicks)
    avgPrice = sumValues.price / float(numTicks)
    print "Max units %i, avg units %f\n" % (maxValuesReduce.units, avgUnits)
    print "Max price %f, min price %f, avg price %f\n"
            % (maxValuesReduce.price, minValuesReduce.price, avgPrice)
### Daily and hourly spreads
# Here is how the daily spread is computed. For each data point, the spread can be calculated
# using the following formula : (ask - bid) / 2 * (ask + bid)
# 1) We have a MapReduce phase that uses the generateSpreadsDailyKeys() function as an argument
# to map(), and the spreadsSumReduce() function as an argument to reduce()
# - The keys will be a unique date in the ISO 8601 format (so that sorting dates
# alphabetically will sort them chronologically)
# - The values will be tuples that contain adequates values to (1) only take one value into
# account per second (which value is picked doesn’t matter), (2) sum the spreads for the
# day, and (3) count the number of spread values that have been added.
# 2) We have a Map phase that computes thee average spread using (b) and (c)
# 3) A final Map phase formats the output by producing a string with the following format:
# "<key (date)>, <average_spread>"
# 4) The output is written using .saveAsTextFile("WDC_daily")
    avgDailySpreads = goodTicks.map(generateSpreadsDailyKeys).reduceByKey(spreadsSumReduce); # (1)
    #avgDailySpreads = avgDailySpreads.map(lambda a: ???) # (2)
    avgDailySpreads = avgDailySpreads.map(lambda (key,a): (key,a[1]/a[2]))
    #avgDailySpreads = avgDailySpreads.sortByKey().map(lambda a: ???) # (3)
    avgDailySpreads = avgDailySpreads.sortByKey().map(lambda (key,a): key+","+str(a))
    avgDailySpreads = avgDailySpreads.saveAsTextFile("WDC_daily") # (4)
    # For the hourly spread you only need to change the key. How?
    avgHourlySpreads = goodTicks.map(generateSpreadsHourlyKeys).reduceByKey(spreadsSumReduce); # (1)
    #avgHourlySpreads = avgHourlySpreads.map(lambda a: ???) # (2)
    avgDailySpreads = avgDailySpreads.map(lambda (key,a): (key,a[1]/a[2]))
    #avgHourlySpreads = avgHourlySpreads.sortByKey().map(lambda a: ???) # (3)
    avgDailySpreads = avgDailySpreads.sortByKey().map(lambda (key,a): key+","+str(a))
    avgHourlySpreads = avgHourlySpreads.saveAsTextFile("WDC_hourly") # (4)
    sc.stop()